# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key (deleted before submission)
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
#load csv from part 1
vacay_load = pd.read_csv('../WeatherPy/finally.csv')
vacay_load

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#configure google maps w g_key (api_key files will be removed)
gmaps.configure(api_key = g_key)

In [ ]:
#create figure and heatmap layer of lat and longitude weighted by humidity
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(vacay_load[['Lat','Lng']], weights = vacay_load['Humidity (%)'])
fig.add_layer(heatmap_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#filtering cities to ones with a max temp betweeen 65 and 86 and a humidity % below 30
hotel_df1 = vacay_load[((vacay_load['Max Temp (F)'] > 65) & (vacay_load['Max Temp (F)'] < 86)) & (vacay_load['Humidity (%)'] < 30)]
hotel_df = hotel_df1.copy().copy()
#empty hotel column
hotel_df['Hotel Name'] = ''
#rearrange columns
hotel_df = hotel_df[['Name','Country','Lat','Lng','Humidity (%)','Wind (mph)','Clouds (%)','Max Temp (F)','Hotel Name','Date']]
hotel_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#create parameter dictionary and save api url into variable
params = {
    'key':g_key,
    'radius':5000,
    'type':'lodging'
}
base = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?{params}'


In [ ]:
#getting coord data from df, searching for hotel, appending selections to list
homaps = []
for city in hotel_df.values:
    params['location'] = (str(city[2:4][0]) + ',' + str(city[2:4][1]))
    
    homaps.append(requests.get(base,params=params).json())

In [ ]:
#extract the first hotel and its name, labeling null if no hotel
hotels = []
for homap in homaps:
    if len(homap['results']) > 0:
        hotels.append(homap['results'][0])
    else:
        hotels.append('Null')
hotel_names = []

for entry in hotels:
    if entry == 'Null':
        hotel_names.append('No Hotel')
        
    else:
        hotel_names.append(entry['name'])

In [ ]:
#adding names to df, removing entries with no hotel
hotel_df['Hotel Name'] = hotel_names
hotel_df = hotel_df.rename(columns={'Name':'City'})
hotel_df = hotel_df[hotel_df['Hotel Name'] != 'No Hotel']
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content =hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig